In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [3]:
class CsvHandler:
    df = None
    quarters = None
    years = None
    max_days = None

    def __init__(self, csv_name: str):
        self.__load_data(csv_name)
        self.df['Norm Adj Close'] = self.__add_normalized_data(self.df)
        self.df['Quarter'] = self.__add_quarters(self.df)
        self.max_days = 252

    def get_equal_length_prices(self, normalized=True):
        df = self.__shift_first_year_prices()
        for i in range(1, len(self.years)):
            df = pd.concat([df, pd.DataFrame(self.get_year_data(year=self.years[i], normalized=normalized))], axis=1)

        df = df[:self.max_days]

        quarters = []
        for j in range(0, len(self.quarters)):
            for i in range(0, self.max_days // 4):
                quarters.append(self.quarters[j])
        quarters = pd.DataFrame(quarters)

        df = pd.concat([df, quarters], axis=1)
        df.columns = self.years + ['Quarter']
        df.index.name = 'Day'

        self.__fill_last_rows(df)

        return df

    def get_year_data(self, year: int, normalized=True):
        if year not in self.years:
            raise ValueError('\n' +
                             'Input year: {} not in available years: {}'.format(year, self.years))

        prices = (self.df.loc[self.df['Date'].dt.year == year])
        if normalized:
            return np.asarray(prices.loc[:, 'Norm Adj Close'])
        else:
            return np.asarray(prices.loc[:, 'Adj Close'])

    def get_whole_prices(self, start_year: int, end_year: int):
        if start_year < self.years[0] or end_year > self.years[-1]:
            raise ValueError('\n' +
                             'Input years out of available range! \n' +
                             'Max range available: {}-{}\n'.format(self.years[0], self.years[-1]) +
                             'Was: {}-{}'.format(start_year, end_year))

        df = (self.df.loc[(self.df['Date'].dt.year >= start_year) & (self.df['Date'].dt.year <= end_year)])
        df = df.loc[:, ['Date', 'Adj Close']]

        return df

    def show(self, max_rows=None, max_columns=None):
        with pd.option_context('display.max_rows', max_rows, 'display.max_columns', max_columns):
            print(self.df)

    def __load_data(self, csv_name: str):
        self.df = pd.read_csv('/content/drive/MyDrive/Intro_DS_2021/Preprocessing Data/' + csv_name + '.csv')
        self.df = self.df.iloc[:, [0, 5]]
        self.df = self.df.dropna()
        self.df.Date = pd.to_datetime(self.df.Date)
        self.quarters = ['Q' + str(i) for i in range(1, 5)]

    def __add_normalized_data(self, df):
        normalized = pd.DataFrame()

        self.years = list(df.Date)
        self.years = list({self.years[i].year for i in range(0, len(self.years))})

        for i in range(0, len(self.years)):
            prices = self.get_year_data(year=self.years[i], normalized=False)
            mean = np.mean(prices)
            std = np.std(prices)
            prices = [(prices[i] - mean) / std for i in range(0, len(prices))]
            prices = [(prices[i] - prices[0]) for i in range(0, len(prices))]
            normalized = normalized.append(prices, ignore_index=True)

        return normalized

    def __add_quarters(self, df):
        quarters = pd.DataFrame()

        for i in range(0, len(self.years)):
            dates = list((df.loc[df['Date'].dt.year == self.years[i]]).iloc[:, 0])
            dates = pd.DataFrame([self.__get_quarter(dates[i].month) for i in range(0, len(dates))])
            quarters = quarters.append(dates, ignore_index=True)

        return quarters

    def __get_quarter(self, month: int):
        return self.quarters[(month - 1) // 3]

    def __shift_first_year_prices(self):
        prices = pd.DataFrame(self.get_year_data(self.years[0]))
        df = pd.DataFrame([0 for _ in range(self.max_days - len(prices.index))])
        df = pd.concat([df, prices], ignore_index=True)

        return df

    def __fill_last_rows(self, df):
        years = self.years[:-1]

        for year in years:
            mean = np.mean(df[year])
            for i in range(self.max_days - 1, -1, -1):
                current_price = df.iloc[i, df.columns.get_loc(year)]
                if np.isnan(current_price):
                    df.iloc[i, df.columns.get_loc(year)] = mean
                else:
                    break

In [4]:
class Plotter:
    __company_name = None
    __company_handler = None
    __prices_data = None
    __quarters = None
    __years = None
    __max_days = None
    __quarter_length = None
    __gpr = None

    def __init__(self, company_name: str):
        self.__company_name = company_name
        self.__company_handler = CsvHandler(company_name)
        self.__prices_data = self.__company_handler.get_equal_length_prices()
        self.__quarters = self.__company_handler.quarters
        self.__years = self.__company_handler.years
        self.__max_days = self.__company_handler.max_days
        self.__quarter_length = int(self.__max_days / 4)
        self.__gpr = Wrapper(company_name)

    def show_preprocessed_price(self, year: int):
        self.show_preprocessed_prices(start_year=year, end_year=year)

    def show_preprocessed_prices(self, start_year: int, end_year: int):
        self.__validate_dates(start_year=start_year, end_year=end_year)

        fig = plt.figure(num=self.__company_name + ' normalized prices')
        ax = plt.gca()
        fig.set_size_inches(12, 6)
        y_low, y_high = 0, 0
        for year in range(start_year, end_year + 1):
            y = self.__prices_data[year]
            if y_low >= min(y):
                y_low = min(y)
            if y_high <= max(y):
                y_high = max(y)
            x = np.linspace(0, len(y), len(y))
            plt.plot(x, y, alpha=.95, label=year)
            plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)

        y_max = max(abs(y_low) - 1, abs(y_high) + 1)
        x_min, x_max = -10, self.__max_days + 10
        ax.set_ylim(bottom=-y_max, top=y_max)
        ax.set_xlim(left=x_min, right=x_max)

        for i in range(0, 5):
            plt.vlines(x=(self.__max_days / 4) * i, ymin=-y_max, ymax=y_max, color='black', linestyles='--', alpha=.6,
                       zorder=-1)
            if i < 4:
                ax.text((self.__max_days / 4) * i + self.__max_days / 8 - 5, y_max - 0.5, self.__quarters[i],
                        fontsize=12)
        plt.hlines(y=0, xmin=x_min, xmax=x_max, color='black', linestyles='--', alpha=.6, zorder=-1)

        plt.grid(True, alpha=.25)
        plt.title(self.__company_name)
        plt.xlabel('Days')
        plt.ylabel('Normalized price')

        plt.tight_layout()

        fname = '{}_{}_{}_normalized_prices.png'.format(self.__company_name, start_year, end_year)
        fig.savefig(fname, dpi=fig.dpi)
        plt.clf()

    def show_gp_prediction(self, train_start: int, train_end: int, pred_year: int, pred_quarters: list = None):
        self.__validate_dates(start_year=train_start, end_year=pred_year)

        prices = self.__prices_data[pred_year]
        prices = prices[prices.iloc[:].notnull()]

        fig = plt.figure(num=self.__company_name + ' prediction')
        ax = plt.gca()
        fig.set_size_inches(12, 6)

        x_obs = list(range(prices.index[0], prices.index[-1] + 1))
        x_mesh, y_mean, y_cov = self.__gpr.get_eval_model(start_year=train_start, end_year=train_end,
                                                          pred_year=pred_year,
                                                          pred_quarters=pred_quarters)
        y_lower = y_mean - np.sqrt(np.diag(y_cov))
        y_upper = y_mean + np.sqrt(np.diag(y_cov))
        y_max = max(abs(min(y_lower) - 1), abs(max(y_upper) + 1))
        ax.set_ylim(bottom=-y_max, top=y_max)

        x_min, x_max = -10, self.__max_days + 10
        ax.set_xlim(left=x_min, right=x_max)

        plt.plot(x_obs, prices, color='#006699', alpha=.95, label=u'Observations ' + str(pred_year), zorder=10)
        plt.plot(x_mesh, y_mean, color='#ff0066', linestyle='--', label=u'Prediction')
        plt.fill_between(x_mesh, y_lower, y_upper,
                         alpha=.25, label='95% confidence', color='#ff0066')

        handles, labels = plt.gca().get_legend_handles_labels()
        new_labels, new_handles = [], []
        for handle, label in zip(handles, labels):
            if label not in new_labels:
                new_labels.append(label)
                new_handles.append(handle)
        plt.legend(new_handles, new_labels, bbox_to_anchor=(0.01, 0.02), loc='lower left', borderaxespad=0.)

        for i in range(0, 5):
            plt.vlines(x=self.__quarter_length * i, ymin=-y_max, ymax=y_max, color='black', linestyles='--', alpha=.6,
                       zorder=-1)
            if i < 4:
                ax.text(self.__quarter_length * i + self.__quarter_length / 2 - 5, y_max - 0.5, self.__quarters[i],
                        fontsize=12)
        plt.hlines(y=0, xmin=x_min, xmax=x_max, color='black', linestyles='--', alpha=.6, zorder=-1)

        plt.grid(True, alpha=.25)
        plt.title(self.__company_name)
        plt.xlabel('Days\n')
        plt.ylabel('Normalized price')

        plt.tight_layout()

        fname = '/content/drive/MyDrive/Intro_DS_2021/Prediction/{}_{}_prediction.png'.format(self.__company_name, pred_year)
        fig.savefig(fname, dpi=fig.dpi)
        plt.clf()

    def show_whole_time_series(self, intermediate: bool = False):
        self.show_time_series(start_year=self.__years[0], end_year=self.__years[-1], intermediate=intermediate)

    def show_time_series(self, start_year: int, end_year: int, intermediate: bool = True):
        self.__validate_dates(start_year=start_year, end_year=end_year)

        prices_data = self.__company_handler.get_whole_prices(start_year=start_year, end_year=end_year)

        fig = plt.figure(num=self.__company_name + ' prices')
        fig.set_size_inches(12, 6)
        plt.plot(prices_data.iloc[:, 0], prices_data.iloc[:, 1], color='#006699', alpha=.95,
                 label=u'Observations ' + str(start_year) + '-' + str(end_year), zorder=10)
        ax = plt.gca()

        x_ticks = []
        for year in range(start_year, end_year + 2):
            if year == end_year + 1:
                current_date = prices_data[prices_data['Date'].dt.year == end_year].iloc[-1, 0]
            else:
                current_date = prices_data[prices_data['Date'].dt.year == year].iloc[0, 0]
            x_ticks.append(current_date)

        x_formatter = mdates.DateFormatter('%d-%m-%Y')
        ax.xaxis.set_major_formatter(x_formatter)
        if not intermediate:
            x_ticks = [x_ticks[0], x_ticks[-2], x_ticks[-1]]
            ax.set_xticks([x_ticks[0], x_ticks[-1]])
        else:
            ax.set_xticks(x_ticks)
        plt.xticks(rotation=20)
        y_min, y_max = ax.get_ylim()
        x_min, x_max = ax.get_xlim()
        ax.set_ylim(bottom=y_min, top=y_max)
        ax.set_xlim(left=x_min, right=x_max)

        for i in range(0, len(x_ticks)):
            plt.vlines(x=x_ticks[i], ymin=y_min, ymax=y_max, color='black', linestyles='--', alpha=.6,
                       zorder=-1)

        plt.grid(True, alpha=0.25)
        plt.legend()
        plt.title(self.__company_name)
        plt.ylabel('Price')

        plt.tight_layout()

        fname = '{}_{}_{}_prices.png'.format(self.__company_name, start_year, end_year)
        fig.savefig(fname, dpi=fig.dpi)
        plt.clf()

    def __validate_dates(self, start_year: int, end_year: int):
        if start_year < self.__years[0] or end_year > self.__years[-1]:
            raise ValueError('\n' +
                             'Input years out of available range! \n' +
                             'Max range available: {}-{}\n'.format(self.__years[0], self.__years[-1]) +
                             'Was: {}-{}'.format(start_year, end_year))

In [5]:
class Wrapper:
    __company_data = None
    __prices_data = None
    __quarters = None
    __max_days = None
    __alpha = None
    __iterations = None
    __kernels = None
    __gp = None

    def __init__(self, company_name: str):
        self.__company_data = CsvHandler(company_name)
        self.__prices_data = self.__company_data.get_equal_length_prices()
        self.__quarters = self.__company_data.quarters
        self.__years = self.__company_data.years
        self.__max_days = self.__company_data.max_days

        kernel = 63 * RBF(length_scale=1)
        self.__alpha = 1e-10
        self.__iterations = 10
        self.__kernels = [kernel]
        self.__gp = GaussianProcessRegressor(kernel=self.__kernels[0], alpha=self.__alpha,
                                             n_restarts_optimizer=self.__iterations,
                                             normalize_y=False)

    def get_eval_model(self, start_year: int, end_year: int, pred_year: int, pred_quarters: list = None):
        years_quarters = list(range(start_year, end_year + 1)) + ['Quarter']
        training_years = years_quarters[:-2]
        df_prices = self.__prices_data[self.__prices_data.columns.intersection(years_quarters)]

        possible_days = list(df_prices.index.values)
        X = np.empty([1,2], dtype=int)
        Y = np.empty([1], dtype=float)

        first_year_prices = df_prices[start_year]
        if start_year == self.__company_data.years[0]:
            first_year_prices = (first_year_prices[first_year_prices.iloc[:] != 0])
            first_year_prices = (pd.Series([0.0], index=[first_year_prices.index[0]-1])).append(first_year_prices)

        first_year_days = list(first_year_prices.index.values)
        first_year_X = np.array([[start_year, day] for day in first_year_days])

        X = first_year_X
        Y = np.array(first_year_prices)
        for current_year in training_years[1:]:
            current_year_prices = list(df_prices.loc[:, current_year])
            current_year_X = np.array([[current_year, day] for day in possible_days])
            X = np.append(X, current_year_X, axis=0)
            Y = np.append(Y, current_year_prices)

        last_year_prices = df_prices[end_year]
        last_year_prices = last_year_prices[last_year_prices.iloc[:].notnull()]

        last_year_days = list(last_year_prices.index.values)
        if pred_quarters is not None:
            length = 63 * (pred_quarters[0] - 1)
            last_year_days = last_year_days[:length]
            last_year_prices = last_year_prices[:length]
        last_year_X = np.array([[end_year, day] for day in last_year_days])

        X = np.append(X, last_year_X, axis=0)
        Y = np.append(Y, last_year_prices)

        if pred_quarters is not None:
            pred_days = [day for day in
                         range(63 * (pred_quarters[0]-1), 63 * pred_quarters[int(len(pred_quarters) != 1)])]
        else:
            pred_days = list(range(0, self.__max_days))
        x_mesh = np.linspace(pred_days[0], pred_days[-1]
                             , 2000)
        x_pred = ([[pred_year, x_mesh[i]] for i in range(len(x_mesh))])

        self.__gp = self.__gp.fit(X, Y)
        self.__kernels.append(self.__gp.kernel_)

        y_mean, y_cov = self.__gp.predict(x_pred, return_cov=True)

        return x_mesh, y_mean, y_cov

    def get_kernels(self):
        return self.__kernels

In [6]:
companies = []
plotters = {}
start_year = 2008

def make_summary(company_name):
    plotter = plotters[company_name]

    plotter.show_whole_time_series()
    plotter.show_time_series(start_year=start_year, end_year=2019)
    plotter.show_preprocessed_prices(start_year=start_year, end_year=2019)
    plotter.show_gp_prediction(train_start=start_year, train_end=2019, pred_year=2020)
    plotter.show_time_series(start_year=start_year, end_year=2021)
    plotter.show_gp_prediction(train_start=start_year, train_end=2021, pred_year=2021, pred_quarters=[3, 4])
    print(company_name + ' summary done!')


def __init_data():
    for company in os.listdir('/content/drive/MyDrive/Intro_DS_2021/Preprocessing Data/'):
        print(company)
        current_company = company.split('.')[0]
        companies.append(current_company)
        plotters[current_company] = (Plotter(company_name=current_company))


def main():
    __init_data()
    for company in companies:
        make_summary(company)
    print("Done!")

In [9]:
main()

PTN_preprocessed.csv
SAVA_preprocessed.csv
IXIC_preprocessed.csv
PTN_preprocessed summary done!
SAVA_preprocessed summary done!
IXIC_preprocessed summary done!
Done!


<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>

<Figure size 864x432 with 0 Axes>